Classical Test Theory with known true score mean, true score variance, and error variance
---

In [2]:
using Turing
using LazyArrays
using StatsPlots
using StatisticalRethinking;
using DataFrames
using Optim
using StatsBase

In [3]:
# Syntax with lazyArrays 
@model function CTT(observations)
    μ_T = 80       # Mean of the true scores
    σ²_T = 36      # Variance of the true scores
    σ²_E = 16      # Variance of the errors 
    τ_T = 1 /σ²_T  # Precision of true scores 
    τ_E = 1 /σ²_E # Precision of the errors 
    T ~ filldist(Normal(μ_T, τ_T), length(observations))
    observations ~ arraydist(LazyArray(@~ Normal.(T,τ_E)))
end

CTT (generic function with 1 method)

- It is said that using LazyArrays is more computationally efficient

In [4]:
# Syntax with lazyArrays 
@model function CTT2(observations)
    μ_T = 80       # Mean of the true scores
    σ²_T = 36      # Variance of the true scores
    σ²_E = 16      # Variance of the errors 
    τ_T = 1 /σ²_T  # Precision of true scores 
    τ_E = 1 /σ²_E # Precision of the errors 
    m,n = size(observations)
    T = Vector{Float64}(undef, m)
    for i in 1:m
            T[i] ~ Normal(μ_T, τ_T)
    end
    
    for i in 1:m
        for j in 1:n
            observations[i,j] ~ Normal(T[i], τ_T)
        end
    end
end

CTT2 (generic function with 1 method)

In [ ]:
ctt_model = CTT([70, 80, 96])
c1 = sample(ctt_model, PG(10), MCMCThreads(),3000,3)
DataFrame(summarystats(c1))

In [31]:
[80 77 80; 83 10 10]

2×3 Matrix{Int64}:
 80  77  80
 83  10  10

In [114]:
MLE_est= optimize(ctt_model, MLE())

ModeResult with maximized lp of 5.56
3-element Named Vector{Float64}
A              │ 
───────────────┼─────
Symbol("T[1]") │ 70.0
Symbol("T[2]") │ 80.0
Symbol("T[3]") │ 96.0

In [106]:
optimize(ctt_model, MAP())

ModeResult with maximized lp of -38038.07
3-element Named Vector{Float64}
A              │ 
───────────────┼────────
Symbol("T[1]") │ 78.3505
Symbol("T[2]") │    80.0
Symbol("T[3]") │ 82.6392

In [ ]:
@model function CTT2(observations)
    μ_T = 80       # Mean of the true scores
    σ²_T = 36      # Variance of the true scores
    σ²_E = 16      # Variance of the errors 
    τ_T = 1 /σ²_T  # Precision of true scores 
    τ_E = 1 /σ²_E # Precision of the errors 
    m,n = size(observations)
    T = Vector{Float64}(undef, m)
    for i in 1:m
            T[i] ~ Normal(μ_T, τ_T)
    end
    
    for i in 1:m
        for j in 1:n
            observations[i,j] ~ Normal(T[i], τ_T)
        end
    end
end

ctt_model = fasterCTT2(obs)
c1 = sample(ctt_model, MH(), MCMCThreads(),3000,10)
plot(c1)
DataFrame(summarystats(c1))

obs = [ 80 77 80 73 73;
        83 79 78 78 77;
        85 77 88 81 80;
        76 76 76 78 67;
        70 69 73 71 77;
        87 89 92 91 87;
        76 75 79 80 75;
        86 75 80 80 82;
        84 79 79 77 82;
        96 85 91 87 90]


        @model function fastCTT2(observations)
            μ_T = 80       # Mean of the true scores
            σ²_T = 36      # Variance of the true scores
            σ²_E = 16      # Variance of the errors 
            τ_T = 1 /σ²_T  # Precision of true scores 
            τ_E = 1 /σ²_E # Precision of the errors 
            m,n = size(observations)
            T ~ filldist(Normal(μ_T, τ_T), m)
            
            observations ~ arraydist([Normal(T[i], τ_T) for i in 1:m])
        end

        @model function fasterCTT2(observations)
            μ_T = 80       # Mean of the true scores
            σ²_T = 36      # Variance of the true scores
            σ²_E = 16      # Variance of the errors 
            τ_T = 1 /σ²_T  # Precision of true scores 
            τ_E = 1 /σ²_E # Precision of the errors 
            m,n = size(observations)
            T ~ filldist(Normal(μ_T, τ_T), m)
            
            observations ~ arraydist(LazyArray(Base.broadcasted((m, s) -> Normal(m, s), T, τ_T)))
        end